In [1]:
import pandas as pd
import numpy  as np

In [3]:
artist_like_df = pd.read_csv('./outputs/artist_like_df.csv')
track_info_df = pd.read_csv('./outputs/track_info_df.csv')

In [4]:
unique_artists = np.unique(np.concatenate((artist_like_df['ARTIST_ID'].values,track_info_df['ARTIST_ID'].values),0))
print('number of total unique artists:',unique_artists.shape[0])

number of total unique artists: 16045


In [5]:
artist_info = pd.DataFrame(columns=['ARTIST_ID','noLikes','noTracks','noAlbums'])

In [6]:
artist_info['ARTIST_ID'] = unique_artists

In [7]:
gp = artist_like_df.groupby(['ARTIST_ID'])
artist_like_gp = gp.apply(lambda x: len(x))
for index,row in artist_info.iterrows():
    try:
        artist_info.loc[index,'noLikes'] = artist_like_gp.loc[row['ARTIST_ID']]
    except(KeyError):
        pass

In [8]:
gp = track_info_df.loc[~track_info_df['ARTIST_ID'].isna()].groupby(['ARTIST_ID','TRACK_ID'])
artist_track_gp = gp.apply(lambda x: len(x))
# print(artist_track_gp.head())
for index,row in artist_info.iterrows():
    try:
        artist_info.loc[index,'noTracks'] = len(artist_track_gp.loc[row['ARTIST_ID']])
    except:
        pass

In [9]:
gp = track_info_df.loc[~track_info_df['ARTIST_ID'].isna()].groupby(['ARTIST_ID','ALBUM_ID'])
artist_album_gp = gp.apply(lambda x: len(x))
for index,row in artist_info.iterrows():
    try:
        artist_info.loc[index,'noAlbums'] = len(artist_album_gp.loc[row['ARTIST_ID']])
    except:
        pass

In [10]:
artist_info.isna().sum()

ARTIST_ID       0
noLikes      8677
noTracks      136
noAlbums      137
dtype: int64

In [11]:
artist_info.head()

,ARTIST_ID,noLikes,noTracks,noAlbums
0,2804986,229,77,21
1,2804988,74,5242,550
2,2804990,2512,246,56
3,2804992,1348,136,36
4,2804994,31,28,3


In [12]:
artist_info.to_csv('./outputs/artist_info.csv')

### -----------------------------------------------

In [17]:
track_like_df= pd.read_csv('./outputs/track_like_expanded.csv',index_col=0)
track_download_df = pd.read_csv('./outputs/track_download_expanded.csv',index_col=0)
track_tag_df = pd.read_csv('./outputs/track_tag_df.csv',index_col=0)
track_purchase_df = pd.read_csv('./outputs/track_purchase_expanded.csv',index_col=0)

/Users/macbookpro/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/macbookpro/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [18]:
print(track_purchase_df.shape)
track_purchase_df.head()

(15214284, 3)


,C_DATE,TRACK_ID,USER_ID
0,2013-04-03 00:04:30,2840546.0,3602578
1,2013-04-03 00:04:30,2840551.0,3602578
2,2013-04-03 00:04:30,2840554.0,3602578
3,2013-04-03 00:04:30,2840555.0,3602578
4,2013-04-03 00:04:30,2839044.0,3602578


In [19]:
unique_tracks = np.unique(np.concatenate((track_purchase_df['TRACK_ID'].values.astype('int64'),track_download_df['TRACK_ID'].values,
                                          track_info_df['TRACK_ID'].values,track_like_df['TRACK_ID'].values,track_tag_df['TRACK_ID'].values),0))
print('number of total unique tracks:',unique_tracks.shape[0])

number of total unique tracks: 125029


In [23]:
track_info = pd.DataFrame(columns=['TRACK_ID','noDownloads','noPurchases','noLikes'])

In [24]:
track_info['TRACK_ID'] = unique_tracks

In [25]:
gp = track_like_df.groupby(['TRACK_ID'])
track_like_gp = gp.apply(lambda x: len(x))
track_info["noLikes"] = track_info.set_index('TRACK_ID')['noLikes'].fillna(track_like_gp).values

In [28]:
gp = track_download_df.groupby(['TRACK_ID'])
track_download_gp = gp.apply(lambda x: len(x))
track_info["noDownloads"] = track_info.set_index('TRACK_ID')['noDownloads'].fillna(track_download_gp).values

In [29]:
gp = track_purchase_df.groupby(['TRACK_ID'])
track_purchase_gp = gp.apply(lambda x: len(x))
track_info["noPurchases"] = track_info.set_index('TRACK_ID')['noPurchases'].fillna(track_purchase_gp).values

In [30]:
track_info.head()

,TRACK_ID,noDownloads,noPurchases,noLikes
0,2828445,92.0,71.0,8.0
1,2828446,92.0,63.0,13.0
2,2828447,92.0,58.0,7.0
3,2828448,92.0,56.0,8.0
4,2828449,92.0,62.0,6.0


In [31]:
track_info.isna().sum()

TRACK_ID           0
noDownloads    25197
noPurchases    31877
noLikes        36459
dtype: int64

In [35]:
track_info.describe()

,TRACK_ID,noDownloads,noPurchases,noLikes
count,1.250290e+05,99832.000000,93152.000000,88570.000000
mean,2.479358e+08,374.200296,163.327508,19.543322
std,2.389391e+08,1787.262460,847.408874,80.090191
min,2.828445e+06,1.000000,1.000000,1.000000
25%,2.859703e+06,23.000000,13.000000,3.000000
50%,1.560569e+08,59.000000,33.000000,6.000000
75%,4.879680e+08,163.000000,87.000000,12.000000
max,5.578127e+08,35285.000000,33146.000000,2787.000000


In [36]:
track_info.to_csv('./outputs/track_info.csv')